# Create SRA submission sheet
This Python Jupyter notebook creates a BioSample submission sheet for the Sequence Read Archive:

First, import Python modules:

In [1]:
import datetime
import glob
import itertools
import os

import natsort

import pandas as pd

import yaml

Display full columns of data frames:

In [2]:
pd.set_option('display.max_colwidth', None)

Read the configuration for the uploads:

In [3]:
with open('upload_config.yaml') as f:
    config = yaml.safe_load(f)

Read the Illumina runs to submit:

In [4]:
print(f"Reading Illumina runs from {config['barcode_runs']}")

illumina_runs = pd.read_csv(config['barcode_runs'])

print('Here are the first few entries in the Illumina runs:')
illumina_runs.head()

Reading Illumina runs from ./barcode_runs_upload.csv
Here are the first few entries in the Illumina runs:


,library,sample,sample_type,sort_bin,concentration,date,number_cells,R1
0,lib1,SortSeq_bin1,SortSeq,1,NaN,200908,1440000,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib1_RBDbin1_S73_R1_001.fastq.gz
1,lib1,SortSeq_bin2,SortSeq,2,NaN,200908,2625000,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib1_RBDbin2_S74_R1_001.fastq.gz
2,lib1,SortSeq_bin3,SortSeq,3,NaN,200908,2370000,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib1_RBDbin3_S75_R1_001.fastq.gz
3,lib1,SortSeq_bin4,SortSeq,4,NaN,200908,2047000,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib1_RBDbin4_S76_R1_001.fastq.gz
4,lib2,SortSeq_bin1,SortSeq,1,NaN,200908,1430000,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib2_RBDbin1_S77_R1_001.fastq.gz


Next make submission entries for the barcode runs, initially in "tidy" format with one FASTQ file per row:

In [5]:
submissions_tidy = (
    illumina_runs
    .assign(
        biosample_accession=lambda x: config['biosample_accession'],
        library_ID=lambda x: x['library'] + '_' + x['sample'],
        title=lambda x: 'Illumina barcode sequencing for ' + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='Illumina HiSeq 2500',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename))
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

assert submissions_tidy['filename_fullpath'].map(os.path.isfile).all()

print('Here are the first few submission entries in tidy format:')
submissions_tidy.head()

Here are the first few submission entries in tidy format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,filename
0,SAMN20174027,lib1_SortSeq_bin1,Illumina barcode sequencing for SortSeq_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib1_RBDbin1_S73_R1_001.fastq.gz,200908_lib1_RBDbin1_S73_R1_001.fastq.gz
1,SAMN20174027,lib1_SortSeq_bin2,Illumina barcode sequencing for SortSeq_bin2,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib1_RBDbin2_S74_R1_001.fastq.gz,200908_lib1_RBDbin2_S74_R1_001.fastq.gz
2,SAMN20174027,lib1_SortSeq_bin3,Illumina barcode sequencing for SortSeq_bin3,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib1_RBDbin3_S75_R1_001.fastq.gz,200908_lib1_RBDbin3_S75_R1_001.fastq.gz
3,SAMN20174027,lib1_SortSeq_bin4,Illumina barcode sequencing for SortSeq_bin4,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib1_RBDbin4_S76_R1_001.fastq.gz,200908_lib1_RBDbin4_S76_R1_001.fastq.gz
4,SAMN20174027,lib2_SortSeq_bin1,Illumina barcode sequencing for SortSeq_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/200917_D00300_1054_AHH332BCX3/Unaligned/Project_tstarr/200908_lib2_RBDbin1_S77_R1_001.fastq.gz,200908_lib2_RBDbin1_S77_R1_001.fastq.gz


For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [6]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

,biosample_accession,library_ID,n_files
0,SAMN20174027,lib1_RaACE2.787.pool6_01_bin1,1
1,SAMN20174027,lib2_RaACE2.9479_05_bin4,1
2,SAMN20174027,lib2_RaACE2.9479_06_bin2,1
3,SAMN20174027,lib2_RaACE2.9479_06_bin3,1
4,SAMN20174027,lib2_RaACE2.9479_06_bin4,1
...,...,...,...
663,SAMN20174027,lib1_cvACE2_04_bin4,1
664,SAMN20174027,lib1_cvACE2_05_bin1,1
665,SAMN20174027,lib1_cvACE2_05_bin2,1
666,SAMN20174027,lib1_cvACE2_05_bin3,1


Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [7]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

print('Here are the first few submission entries in wide format:')
submissions_wide.head()

Here are the first few submission entries in wide format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename
0,SAMN20174027,lib1_SortSeq_bin1,Illumina barcode sequencing for SortSeq_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,200908_lib1_RBDbin1_S73_R1_001.fastq.gz
1,SAMN20174027,lib1_SortSeq_bin2,Illumina barcode sequencing for SortSeq_bin2,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,200908_lib1_RBDbin2_S74_R1_001.fastq.gz
2,SAMN20174027,lib1_SortSeq_bin3,Illumina barcode sequencing for SortSeq_bin3,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,200908_lib1_RBDbin3_S75_R1_001.fastq.gz
3,SAMN20174027,lib1_SortSeq_bin4,Illumina barcode sequencing for SortSeq_bin4,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,200908_lib1_RBDbin4_S76_R1_001.fastq.gz
4,SAMN20174027,lib2_SortSeq_bin1,Illumina barcode sequencing for SortSeq_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,200908_lib2_RBDbin1_S77_R1_001.fastq.gz


Now write the wide submissions data frame to a `*.tsv` file that can be used for uploading the SRA submission website:

In [8]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

We also want to write a file with all of the FASTQ files that are in the submission spreadsheet:

In [9]:
fastq_file_list = 'FASTQs_to_upload.csv'

print(f"Writing list of all the FASTQ files to upload to {fastq_file_list}")

(submissions_tidy
 [['filename_fullpath', 'filename']]
 .to_csv(fastq_file_list, index=False)
 )

Writing list of all the FASTQ files to upload to FASTQs_to_upload.csv
